In [1]:
#Libraries
#Dataframe and arrays
import pandas as pd
import numpy as np

#Graphing/ Charting/ Visualisations
import matplotlib as plt

In [2]:
df = pd.read_excel('../data/online_fashion.xlsx')

In [3]:
df_raw = df

In [4]:
#Find & Remove Strange StockCodes
df_unique_stock_codes = df['StockCode'].unique()
df_unique_stock_codes = pd.DataFrame(df_unique_stock_codes)
df_unique_stock_codes['len'] = df_unique_stock_codes[0].astype(str).str.len()
df_unique_stock_codes.rename(columns={0:'StockCode'}, inplace=True)
df_unique_stock_codes.sort_values(by='len')
df_unique_stock_codes['len'] = df_unique_stock_codes['len'].astype(int)
df_stock_to_drop_stg = df_unique_stock_codes.loc[(df_unique_stock_codes['len'] >= 8) | (df_unique_stock_codes['len'] <= 4)].sort_values(by='len')
df_stock_to_drop_stg.reset_index(drop=True)
df_stock_to_drop = pd.concat([df_stock_to_drop_stg.iloc[0:10], df_stock_to_drop_stg.iloc[-6:], df_stock_to_drop_stg.iloc[-9:-8] ])
df_stock_to_drop

df = df[~(df['StockCode'].isin(df_stock_to_drop.StockCode) )]

df.shape

(538959, 8)

In [5]:
df['Month'] = df['InvoiceDate'].dt.month_name()

C:\Users\ellio\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [6]:
#Identify if Return(1) or Sale(0)
def transtype(row): 
    """
    Input:
    Output:
    
    The purpose of this function is to .....
    """
    if row['Quantity'] <= 0 : 
        return 1
    else: 
        return 0

df['Return'] = df.apply(transtype, axis=1)
df.head()

C:\Users\ellio\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Return
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,December,0
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,0
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,December,0
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,0
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,0


In [7]:
df['Rev'] = (df.Quantity * df.UnitPrice)
df.head()

C:\Users\ellio\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Return,Rev
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,December,0,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,0,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,December,0,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,0,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,0,20.34


In [8]:
#Number of rows with no CustomerID
len(df[df['CustomerID'].isnull()]) #134,050

134050

In [9]:
#Number of items with no CustomerID
df[df['CustomerID'].isnull()].Quantity.sum() #264,390

264390

In [10]:
#Number of items sold with no CustomerID
df[(df['CustomerID'].isnull() &  (df.Return == 0) )].Quantity.sum() #47,3893

473893

In [11]:
#Number of items returned with no CustomerID
df[(df['CustomerID'].isnull() &  (df.Return == 1) )].Quantity.sum() #-209,503

-209503

In [12]:
#Value of items sold with no CustomerID
df[(df['CustomerID'].isnull() &  (df.Return == 0) )].Rev.sum() #1,510,366.41

1510366.41

In [13]:
#Value of items returned with no CustomerID
df[(df['CustomerID'].isnull() &  (df.Return == 1) )].Rev.sum() -4,320.85

(-4324.85, 320.85)

In [14]:
df.dtypes

InvoiceNo              object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
UnitPrice             float64
CustomerID            float64
Country                object
Month                  object
Return                  int64
Rev                   float64
dtype: object

In [15]:
#Number of items returned with no CustomerID in May
df[(df['CustomerID'].isnull() &  (df.Return == 1) & (df.Month == 'May') )].Quantity.sum() #13,172

-13172

In [16]:
#Vale of items returned with no CustomerID in May
df[(df['CustomerID'].isnull() &  (df.Return == 1) & (df.Month == 'May') )].Rev.sum() #-19.98

-19.98

In [17]:
#Number of items returned with no CustomerID in June
df[(df['CustomerID'].isnull() &  (df.Return == 1) & (df.Month == 'June') )].Quantity.sum() #45,917

-45917

In [18]:
#Number of items returned with no CustomerID in June
df[(df['CustomerID'].isnull() &  (df.Return == 1) & (df.Month == 'June') )].Rev.sum() #-1,108.13

-1108.13

In [19]:
df_awp_stg = df[(df['CustomerID'] > 0)]

In [20]:
df_awp = df_awp_stg.groupby('StockCode').agg({'Rev': ['sum'], 'Quantity':['sum']})
df_awp = df_awp.reset_index(drop=False)
df_awp.columns = ['StockCode','Rev','Quantity']
df_awp['AWP'] = df_awp.Rev / df_awp.Quantity
df_awp.head()

,StockCode,Rev,Quantity,AWP
0,10002,699.55,823,0.850000
1,10080,114.41,291,0.393162
2,10120,40.53,193,0.210000
3,10125,930.30,1226,0.758809
4,10133,1139.41,2374,0.479954


In [21]:
#Working version
def awp_lookup(row):
    """
    Input: StockCode taken from row within dataframe
    Output: The corresponding AWP of that item
    
    The purpose of this function is to vlookup value of item from a reference dataframe (df_awp)
    """
    if row is not None:
        try:
            awp = df_awp.loc[(df_awp['StockCode'] == row.StockCode)]["AWP"]
            return awp
        except:
            return 0
    else:
        return 0

In [22]:
df_returns = df[  (df.Return == 1) ]

In [23]:
df_awp = df_awp_stg.groupby('StockCode').agg({'Rev': ['sum'], 'Quantity':['sum']})
df_awp = df_awp.reset_index(drop=False)
df_awp.columns = ['StockCode','Rev','Quantity']
df_awp['AWP'] = df_awp.Rev / df_awp.Quantity
df_awp.head()





,StockCode,Rev,Quantity,AWP
0,10002,699.55,823,0.850000
1,10080,114.41,291,0.393162
2,10120,40.53,193,0.210000
3,10125,930.30,1226,0.758809
4,10133,1139.41,2374,0.479954


In [24]:
df_returns.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Return,Rev
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,2010-12-01 09:49:00,4.65,15311.0,United Kingdom,December,1,-4.65
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,2010-12-01 10:24:00,1.65,17548.0,United Kingdom,December,1,-19.80
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,December,1,-6.96
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,December,1,-6.96
238,C536391,21980,PACK OF 12 RED RETROSPOT TISSUES,-24,2010-12-01 10:24:00,0.29,17548.0,United Kingdom,December,1,-6.96


In [25]:
df_returns = df_returns.groupby('StockCode').agg({'Rev': ['sum'], 'Quantity':['sum']})
df_returns = df_returns.reset_index(drop=False)


In [26]:
df_returns.columns = ['StockCode','Rev','Quantity']
df_returns.head()

,StockCode,Rev,Quantity
0,10002,0.00,-3
1,10133,-4.20,-92
2,10134,0.00,-19
3,10135,-1.25,-1
4,11001,-237.05,-185


In [27]:
df_awp.head()

,StockCode,Rev,Quantity,AWP
0,10002,699.55,823,0.850000
1,10080,114.41,291,0.393162
2,10120,40.53,193,0.210000
3,10125,930.30,1226,0.758809
4,10133,1139.41,2374,0.479954


In [28]:
#df_returns = df_returns.join(df_awp, lsuffix='StockCode', rsuffix='StockCode')
df_returns = pd.merge(df_returns, df_awp, how='left', left_on='StockCode', right_on='StockCode')
df_returns.head()

,StockCode,Rev_x,Quantity_x,Rev_y,Quantity_y,AWP
0,10002,0.00,-3,699.55,823.0,0.850000
1,10133,-4.20,-92,1139.41,2374.0,0.479954
2,10134,0.00,-19,NaN,NaN,NaN
3,10135,-1.25,-1,1785.44,1937.0,0.921755
4,11001,-237.05,-185,1715.95,1067.0,1.608201


In [29]:
df_returns.head()


,StockCode,Rev_x,Quantity_x,Rev_y,Quantity_y,AWP
0,10002,0.00,-3,699.55,823.0,0.850000
1,10133,-4.20,-92,1139.41,2374.0,0.479954
2,10134,0.00,-19,NaN,NaN,NaN
3,10135,-1.25,-1,1785.44,1937.0,0.921755
4,11001,-237.05,-185,1715.95,1067.0,1.608201


In [30]:
df_returns['ActRefundGBP'] = df_returns['Quantity_x']*df_returns['AWP']

In [31]:
df_returns.head()

,StockCode,Rev_x,Quantity_x,Rev_y,Quantity_y,AWP,ActRefundGBP
0,10002,0.00,-3,699.55,823.0,0.850000,-2.550000
1,10133,-4.20,-92,1139.41,2374.0,0.479954,-44.155737
2,10134,0.00,-19,NaN,NaN,NaN,NaN
3,10135,-1.25,-1,1785.44,1937.0,0.921755,-0.921755
4,11001,-237.05,-185,1715.95,1067.0,1.608201,-297.517104


In [32]:
df_returns['ActRefundGBP'].sum()

-inf

In [33]:
df_returns['ActRefundGBP'].sort_values()

1964            -inf
1473   -8.570707e+04
2219   -3.265650e+04
2152   -3.133939e+04
1053   -1.654703e+04
2217   -1.357425e+04
1052   -1.246267e+04
1430   -1.109490e+04
902    -1.046405e+04
1356   -7.812775e+03
2431   -7.786722e+03
52     -7.596446e+03
1434   -7.132950e+03
1431   -6.848110e+03
1432   -6.845878e+03
1433   -6.552000e+03
966    -5.818589e+03
190    -4.868209e+03
2196   -4.625068e+03
411    -4.603536e+03
224    -4.590601e+03
1796   -4.413274e+03
1193   -4.036395e+03
539    -4.000119e+03
412    -3.984875e+03
800    -3.950444e+03
2195   -3.809836e+03
1354   -3.780250e+03
1808   -3.759505e+03
1612   -3.729355e+03
            ...     
2353             NaN
2367             NaN
2368             NaN
2369             NaN
2370             NaN
2371             NaN
2392             NaN
2447             NaN
2456             NaN
2469             NaN
2478             NaN
2494             NaN
2497             NaN
2500             NaN
2512             NaN
2515             NaN
2517         

In [34]:
df_returns.iloc[1964]

StockCode       85047
Rev_x          -18.25
Quantity_x         -3
Rev_y             0.8
Quantity_y          0
AWP               inf
ActRefundGBP     -inf
Name: 1964, dtype: object

In [35]:
df_awp[df_awp['StockCode'] == '85047']

,StockCode,Rev,Quantity,AWP


In [36]:
df[df['StockCode'] == '85047']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Return,Rev


In [37]:
df_raw[df_raw['StockCode'] == '85047']

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


In [38]:
df_awp[df_awp['StockCode'] == '85047']

,StockCode,Rev,Quantity,AWP


In [39]:
df_returns[df_returns['StockCode'] == '85047']

,StockCode,Rev_x,Quantity_x,Rev_y,Quantity_y,AWP,ActRefundGBP


In [40]:
df_returnsB = df_returns.drop(1964)

In [41]:
df_returnsB.head()

,StockCode,Rev_x,Quantity_x,Rev_y,Quantity_y,AWP,ActRefundGBP
0,10002,0.00,-3,699.55,823.0,0.850000,-2.550000
1,10133,-4.20,-92,1139.41,2374.0,0.479954,-44.155737
2,10134,0.00,-19,NaN,NaN,NaN,NaN
3,10135,-1.25,-1,1785.44,1937.0,0.921755,-0.921755
4,11001,-237.05,-185,1715.95,1067.0,1.608201,-297.517104


In [42]:
df_returnsB.ActRefundGBP.sum()

-582089.2195770355

In [43]:
df_raw.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [44]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Month,Return,Rev
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,December,0,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,0,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,December,0,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,0,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,December,0,20.34


In [48]:
df[df['Return'] ==1 ].Rev.sum()

-478724.18000000005